In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import numpy as np
data_dir = 'gdrive/My Drive/EEG_project/'
X_test = np.load(data_dir + "X_test.npy")
y_test = np.load(data_dir + "y_test.npy")
person_train_valid = np.load(data_dir + "person_train_valid.npy")
X_train_valid = np.load(data_dir + "X_train_valid.npy")
y_train_valid = np.load(data_dir + "y_train_valid.npy")
person_test = np.load(data_dir +"person_test.npy")

Mounted at /content/gdrive


In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
for i in range(len(y_train_valid)):
    if(y_train_valid[i] == 769):
        y_train_valid[i] = 0
    if(y_train_valid[i] == 770):
        y_train_valid[i] = 1
    if(y_train_valid[i] == 771):
        y_train_valid[i] = 2
    if(y_train_valid[i] == 772):
        y_train_valid[i] = 3

In [0]:
for i in range(len(y_test)):
    if(y_test[i] == 769):
        y_test[i] = 0
    if(y_test[i] == 770):
        y_test[i] = 1
    if(y_test[i] == 771):
        y_test[i] = 2
    if(y_test[i] == 772):
        y_test[i] = 3

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Temporal Convolution
        self.conv1 = nn.Conv2d(1, 40, (1, 40), stride = (1, 2))
        nn.init.xavier_uniform_(self.conv1.weight)
        # Batch Normalization
        self.batch1 = nn.BatchNorm2d(40)
        # ELU
        self.elu1 = nn.ELU()
        # Spatial Convolution
        self.conv2 = nn.Conv2d(40, 40, (22, 1), stride = (1, 1))
        nn.init.xavier_uniform_(self.conv2.weight)
        # Batch Normalization
        self.batch2 = nn.BatchNorm2d(40)
        # ELU
        self.elu2 = nn.ELU()
        # Avg Pooling
        self.pool1 = nn.AvgPool2d((1, 75), stride = (1, 15))
        # Dropout
        self.dropout = nn.Dropout(p = 0.3)
        # Fully Connected
        self.fc1 = nn.Linear(40 * 28, 9)
        # Softmax
        self.softmax = nn.Softmax(1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.elu1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.elu2(x)
        x = self.pool1(x)
        x = self.dropout(x)
        x = x.reshape(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [0]:
cnn = CNN()

In [0]:
lr = 0.005
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr)

In [0]:
person_train_valid_reshape = []
person_test_reshape = []
for i in range(len(person_train_valid)):
    person_train_valid_reshape = np.concatenate((person_train_valid_reshape, person_train_valid[i]))
for i in range(len(person_test)):
    person_test_reshape = np.concatenate((person_test_reshape, person_test[i]))
person_train_valid_reshape = person_train_valid_reshape.astype(int)   

In [10]:
print(person_train_valid_reshape.astype(int))
print(y_train_valid)

[0 0 0 ... 8 8 8]
[2 3 0 ... 3 3 2]


In [0]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

In [12]:
batch_size = 500
num_iters = 1
num_epoch = 200
epoch_loss = [0.0 for i in range(num_epoch)]
for epoch in range(num_epoch):
    counter = 0
    for train_index, valid_index in kf.split(X_train_valid):
        X_train_cur = torch.tensor(X_train_valid[train_index].reshape(X_train_valid[train_index].shape[0], -1, 22, 1000), dtype=torch.float)
        X_valid_cur = torch.tensor(X_train_valid[valid_index].reshape(X_train_valid[valid_index].shape[0], -1, 22, 1000), dtype=torch.float)
        y_train_cur = torch.tensor(person_train_valid_reshape[train_index], dtype=torch.long)
        y_valid_cur = torch.tensor(person_train_valid_reshape[valid_index], dtype=torch.long)
        for it in np.arange(num_iters):
            index = np.random.choice(X_train_cur.shape[0], batch_size)
            X_batch = X_train_cur[index, :, :, :]
            y_batch = y_train_cur[index]
            optimizer.zero_grad()
            outputs = cnn(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss[epoch] += loss
            counter += 1
        # Validation Accuracy for current K-fold
        output_val = cnn(X_valid_cur)
        prediction_val = torch.argmax(output_val, axis = 1)
        accuracy_val = float(prediction_val.eq(y_valid_cur).sum()) / float(y_valid_cur.shape[0])
        print('Current Epoch: {}, Validation Accuracy: {}'.format(epoch, accuracy_val))
    epoch_loss[epoch] /= counter 
    print('Current Epoch: {}, Epoch Loss: {}'.format(epoch, epoch_loss[epoch]))
        
        

Current Epoch: 0, Validation Accuracy: 0.0
Current Epoch: 0, Validation Accuracy: 0.001890359168241966
Current Epoch: 0, Validation Accuracy: 0.14177693761814744
Current Epoch: 0, Validation Accuracy: 0.5492424242424242
Current Epoch: 0, Epoch Loss: 2.166403293609619
Current Epoch: 1, Validation Accuracy: 0.00945179584120983
Current Epoch: 1, Validation Accuracy: 0.3837429111531191
Current Epoch: 1, Validation Accuracy: 0.502835538752363
Current Epoch: 1, Validation Accuracy: 0.5132575757575758
Current Epoch: 1, Epoch Loss: 2.005845308303833
Current Epoch: 2, Validation Accuracy: 0.17580340264650285
Current Epoch: 2, Validation Accuracy: 0.29867674858223064
Current Epoch: 2, Validation Accuracy: 0.7088846880907372
Current Epoch: 2, Validation Accuracy: 0.6647727272727273
Current Epoch: 2, Epoch Loss: 1.904441475868225
Current Epoch: 3, Validation Accuracy: 0.22873345935727787
Current Epoch: 3, Validation Accuracy: 0.6502835538752363
Current Epoch: 3, Validation Accuracy: 0.640831758034

In [13]:
X_test = torch.tensor(X_test.reshape(X_test.shape[0], -1, 22, 1000), dtype= torch.float)
y_test = torch.tensor(person_test_reshape, dtype = torch.long)
output_test = cnn(X_test)
prediction_test = torch.argmax(output_test, axis = 1)
accuracy_test = float(prediction_test.eq(y_test).sum()) / float(person_test_reshape.shape[0])
print('Current Epoch: {}, Test Accuracy: {}'.format(epoch, accuracy_test))

Current Epoch: 199, Test Accuracy: 0.9932279909706546


In [14]:
torch.save(cnn, 'model.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
